This notebook is aiming at converting the label data to structured format. This notebook also checks the distribution of labels.

Prerequisite:

1) .xlsx format label file stored locally

2) Understanding of the label files. In this example, labels are in column "User entered Pages"

Output:

All reformat labels will be stored in one csv file "batch3_label_revised.csv" and it will be saved locally

In [1]:
# Import required libraries
import pandas as pd
import numpy as np

In [2]:
# Specify label .xlsx filename and location
key_file_addr = "Batch3Label.xlsx"

In [3]:
# Load label file
key_file = pd.read_excel(key_file_addr)
key_file

CR_ID                                   CRFileName        CRFieldName  \
0        1382    OCR_1382_1003700000000633142_11068501.pdf    admission_order   
1        1382    OCR_1382_1003700000000633142_11068501.pdf  discharge_summary   
2        1382    OCR_1382_1003700000000633142_11068501.pdf   history_physical   
3        5217    OCR_5217_1003700000000642572_11072823.pdf         admit_type   
4        7132    OCR_7132_1003700000000680026_11074907.pdf  discharge_summary   
...       ...                                          ...                ...   
35140  112864  OCR_112864_1003700000000633020_11248108.pdf                mar   
35141  113136  OCR_113136_1003700000000697811_11249401.pdf   history_physical   
35142  113136  OCR_113136_1003700000000697811_11249401.pdf               labs   
35143  113136  OCR_113136_1003700000000697811_11249401.pdf                mar   
35144  114686  OCR_114686_1003700000000675009_11251999.pdf   history_physical   

      User entered Pages Cannon Identified Pages  
0                     29                       1  
1                    NaN                     NaN  
2                  43-47                   43-47  
3                    2,3                     2,3  
4                262-278            17-59,61-113  
...                  ...                     ...  
35140              12-24           12-24,222-224  
35141              40-46                     NaN  
35142            203-212           58-60,203-212  
35143                NaN                     NaN  
35144              79-85               79-86,173  

[35145 rows x 5 columns]

In [4]:
# Check how many labels are in label file 
label_from_all = key_file['CRFieldName']
label_from_all.unique()

array(['admission_order', 'discharge_summary', 'history_physical',
       'admit_type', 'admission_reason', 'mar', 'labs', 'flowsheets',
       'radiology', 'consult', 'physician_query', 'procedures',
       'progress_notes', 'emergency_department', 'case_management',
       'discharge_disposition', 'nutritional_assessment'], dtype=object)

In [5]:
# Drop empty labels records
valid_label_df = key_file.dropna(subset = ['User entered Pages'])
valid_label_df

CR_ID                                   CRFileName        CRFieldName  \
0        1382    OCR_1382_1003700000000633142_11068501.pdf    admission_order   
2        1382    OCR_1382_1003700000000633142_11068501.pdf   history_physical   
3        5217    OCR_5217_1003700000000642572_11072823.pdf         admit_type   
4        7132    OCR_7132_1003700000000680026_11074907.pdf  discharge_summary   
5        1382    OCR_1382_1003700000000633142_11068501.pdf   admission_reason   
...       ...                                          ...                ...   
35139  112864  OCR_112864_1003700000000633020_11248108.pdf            consult   
35140  112864  OCR_112864_1003700000000633020_11248108.pdf                mar   
35141  113136  OCR_113136_1003700000000697811_11249401.pdf   history_physical   
35142  113136  OCR_113136_1003700000000697811_11249401.pdf               labs   
35144  114686  OCR_114686_1003700000000675009_11251999.pdf   history_physical   

      User entered Pages Cannon Identified Pages  
0                     29                       1  
2                  43-47                   43-47  
3                    2,3                     2,3  
4                262-278            17-59,61-113  
5                 1,4,44               1,4,20,44  
...                  ...                     ...  
35139                  0                   29-30  
35140              12-24           12-24,222-224  
35141              40-46                     NaN  
35142            203-212           58-60,203-212  
35144              79-85               79-86,173  

[29434 rows x 5 columns]

In [6]:
# As each record may contains multiple pages, this does not show the page-level label distribution
valid_label = valid_label_df['CRFieldName']
valid_label.value_counts()

admission_reason          2368
labs                      2360
admission_order           2342
history_physical          2332
discharge_summary         2332
radiology                 2311
progress_notes            2285
mar                       2249
admit_type                2220
consult                   2015
emergency_department      1968
flowsheets                1513
procedures                1320
discharge_disposition      640
physician_query            497
nutritional_assessment     356
case_management            326
Name: CRFieldName, dtype: int64

In [7]:
# Organize training data
df = valid_label_df

##### Main part to reformat labels

This part also removes label record which has page "0". As "0" is not valid page number.


In [8]:
modified_df =  pd.DataFrame()

for i in range(len(df)):
    col = 'User entered Pages'
    filename = df.iloc[i]['CRFileName'].split("_")[-2] + "_" + df.iloc[i]['CRFileName'].split("_")[-1]
    filename_cut = filename.split('_')[-2][-6:]
    print(filename)
    ds =  df.iloc[i][col]
    temp_list = []
    if str(ds).startswith('-'):
        ds = str(ds)[1:]
    if str(ds).endswith('-') :
        ds = str(ds)[:-1]

    try:
        if not pd.isna(df.iloc[i][col]):
            try:
                ds = int(ds)
                # Remove labels with page 0
                if ds == 0:
                    continue
                else:
                    temp_list.append(ds)
            except:
                if (ds.split(',').__len__() > 1):
                    for j in range(len(ds.split(','))):
                        if '-' in ds.split(',')[j].strip():
                            min_no = int(ds.split(',')[j].split('-')[0])
                            max_no = int(ds.split(',')[j].split('-')[1])
                            temp_list.extend(list(range(min_no, max_no + 1)))
                        elif '-' not in ds.split(',')[j].strip():
                            if ds.split(',')[j].strip().split('-') == ['']:
                                pass
                            else:
                                min_no = int(ds.split(',')[j].strip().split('-')[0])
                                temp_list.extend([min_no])
                else:
                    try:
                        for j in range(len(ds.split(','))):
                            min_no = int(ds.split(',')[j].split('-')[0])
                            max_no = int(ds.split(',')[j].split('-')[1])
                            temp_list.extend(list(range(min_no,max_no + 1)))

                        if len(ds.split(',')[j].split('-')) == 1:
                            temp_list.append(int(ds))

                    except Exception as e:
                        print(e)

        modified_df = pd.concat([modified_df, pd.DataFrame.from_dict ({'page':temp_list,'class': df.iloc[i]['CRFieldName'] , 'filename' : filename, 'filename_cut' : filename_cut})], ignore_index = True)
    except Exception as e:
        print(e)
        
# Convert Page to Int
modified_df['page'] = modified_df['page'].astype(int)

1003700000000633142_11068501.pdf
1003700000000633142_11068501.pdf
1003700000000642572_11072823.pdf
1003700000000680026_11074907.pdf
1003700000000633142_11068501.pdf
1003700000000680026_11074907.pdf
1003700000000678763_11076622.pdf
1003700000000633142_11068501.pdf
1003700000000678763_11076622.pdf
1003700000000633142_11068501.pdf
1003700000000633142_11068501.pdf
1003700000000678763_11076622.pdf
1003700000000680010_11073524.pdf
1003700000000680026_11074907.pdf
1003700000000669477_11082117.pdf
1003700000000680026_11074907.pdf
1003700000000680026_11074907.pdf
1003700000000680026_11074907.pdf
1003700000000678763_11076622.pdf
1003700000000669477_11082117.pdf
1003700000000633142_11068501.pdf
1003700000000669477_11082117.pdf
1003700000000674647_11082141.pdf
1003700000000633142_11068501.pdf
1003700000000687369_11068908.pdf
1003700000000674647_11082141.pdf
1003700000000680026_11074907.pdf
1003700000000633142_11068501.pdf
1003700000000669477_11082117.pdf
1003700000000678763_11076622.pdf
1003700000

1003700000000692106_11103169.pdf
1003700000000693568_11102778.pdf
1003700000000689886_11103154.pdf
1003700000000684915_11103123.pdf
1003700000000693568_11102778.pdf
1003700000000684915_11103123.pdf
1003700000000559904_11102968.pdf
1003700000000693568_11102778.pdf
1003700000000697756_11103816.pdf
1003700000000697612_11103216.pdf
1003700000000695713_11103205.pdf
1003700000000695713_11103205.pdf
1003700000000693458_11103189.pdf
1003700000000693458_11103189.pdf
1003700000000693150_11103180.pdf
1003700000000693132_11103178.pdf
1003700000000693132_11103178.pdf
1003700000000692106_11103169.pdf
1003700000000689886_11103154.pdf
1003700000000559904_11102968.pdf
1003700000000559904_11102968.pdf
1003700000000697756_11103816.pdf
1003700000000697160_11103807.pdf
1003700000000697160_11103807.pdf
1003700000000697160_11103807.pdf
1003700000000697160_11103807.pdf
1003700000000680026_11074907.pdf
1003700000000697439_11103810.pdf
1003700000000697439_11103810.pdf
1003700000000697135_11103806.pdf
1003700000

1003700000000652814_11111827.pdf
1003700000000630115_11105445.pdf
1003700000000697579_11103814.pdf
1003700000000689652_11106502.pdf
1003700000000697369_11103809.pdf
1003700000000697579_11103814.pdf
1003700000000689652_11106502.pdf
1003700000000697369_11103809.pdf
1003700000000689652_11106502.pdf
1003700000000689414_11105085.pdf
1003700000000689414_11105085.pdf
1003700000000689414_11105085.pdf
1003700000000688279_11105073.pdf
1003700000000688279_11105073.pdf
1003700000000689652_11106502.pdf
1003700000000688279_11105073.pdf
1003700000000687943_11105071.pdf
1003700000000689414_11105085.pdf
1003700000000696975_11105651.pdf
1003700000000687313_11106488.pdf
1003700000000689414_11105085.pdf
1003700000000687943_11105071.pdf
1003700000000694029_11106171.pdf
1003700000000696975_11105651.pdf
1003700000000697064_11111678.pdf
1003700000000687943_11105071.pdf
1003700000000696975_11105651.pdf
1003700000000697064_11111678.pdf
1003700000000611548_11106312.pdf
1003700000000652814_11111827.pdf
1003700000

1003700000000693580_11116352.pdf
1003700000000693580_11116352.pdf
1003700000000697287_11116366.pdf
1003700000000698702_11116393.pdf
1003700000000700004_11116451.pdf
1003700000000693404_11116350.pdf
1003700000000699980_11116450.pdf
1003700000000543962_11116157.pdf
1003700000000697287_11116366.pdf
1003700000000543962_11116157.pdf
1003700000000697287_11116366.pdf
1003700000000543962_11116157.pdf
1003700000000699980_11116450.pdf
1003700000000697229_11116365.pdf
1003700000000543962_11116157.pdf
1003700000000699723_11116412.pdf
1003700000000698702_11116393.pdf
1003700000000543962_11116157.pdf
1003700000000697130_11116364.pdf
1003700000000699723_11116412.pdf
1003700000000697130_11116364.pdf
1003700000000699723_11116412.pdf
1003700000000697130_11116364.pdf
1003700000000699723_11116412.pdf
1003700000000693404_11116350.pdf
1003700000000697130_11116364.pdf
1003700000000698702_11116393.pdf
1003700000000704595_11116640.pdf
1003700000000704595_11116640.pdf
1003700000000697058_11116363.pdf
1003700000

1003700000000698528_11116970.pdf
1003700000000697700_11116968.pdf
1003700000000694674_11117609.pdf
1003700000000694674_11117609.pdf
1003700000000694674_11117609.pdf
1003700000000694453_11117604.pdf
1003700000000698528_11116970.pdf
1003700000000694674_11117609.pdf
1003700000000694453_11117604.pdf
1003700000000694453_11117604.pdf
1003700000000694453_11117604.pdf
1003700000000704502_11117025.pdf
1003700000000704502_11117025.pdf
1003700000000704474_11117024.pdf
1003700000000704474_11117024.pdf
1003700000000698528_11116970.pdf
1003700000000704474_11117024.pdf
1003700000000702032_11117006.pdf
1003700000000702032_11117006.pdf
1003700000000700905_11116994.pdf
1003700000000617836_11116872.pdf
1003700000000700905_11116994.pdf
1003700000000700905_11116994.pdf
1003700000000698528_11116970.pdf
1003700000000698528_11116970.pdf
1003700000000697700_11116968.pdf
1003700000000694674_11117609.pdf
1003700000000695068_11116963.pdf
1003700000000695068_11116963.pdf
1003700000000695068_11116963.pdf
1003700000

...

1003700000000720169_11239015.pdf
1003700000000720169_11239015.pdf
1003700000000745963_11238709.pdf
1003700000000745973_11238716.pdf
1003700000000740750_11238305.pdf
1003700000000737256_11239038.pdf
1003700000000740716_11238300.pdf
1003700000000721996_11239020.pdf
1003700000000740716_11238300.pdf
1003700000000740716_11238300.pdf
1003700000000740568_11238289.pdf
1003700000000740568_11238289.pdf
1003700000000740367_11238275.pdf
1003700000000740367_11238275.pdf
1003700000000740367_11238275.pdf
1003700000000740367_11238275.pdf
1003700000000739798_11238196.pdf
1003700000000739798_11238196.pdf
1003700000000721780_11239019.pdf
1003700000000746744_11238808.pdf
1003700000000721780_11239019.pdf
1003700000000721780_11239019.pdf
1003700000000720169_11239015.pdf
1003700000000746275_11238755.pdf
1003700000000720169_11239015.pdf
1003700000000746275_11238755.pdf
1003700000000720169_11239015.pdf
1003700000000746196_11238744.pdf
1003700000000720169_11239015.pdf
1003700000000697231_11238996.pdf
1003700000

1003700000000697261_11240797.pdf
1003700000000744588_11241856.pdf
1003700000000697261_11240797.pdf
1003700000000647089_11240750.pdf
1003700000000633495_11240747.pdf
1003700000000744588_11241856.pdf
1003700000000633495_11240747.pdf
1003700000000633495_11240747.pdf
1003700000000744588_11241856.pdf
1003700000000633495_11240747.pdf
1003700000000605654_11240741.pdf
1003700000000605654_11240741.pdf
1003700000000728037_11241768.pdf
1003700000000647089_11240750.pdf
1003700000000744588_11241856.pdf
1003700000000728037_11241768.pdf
1003700000000728037_11241768.pdf
1003700000000691331_11241693.pdf
1003700000000728037_11241768.pdf
1003700000000691331_11241693.pdf
1003700000000691331_11241693.pdf
1003700000000744588_11241856.pdf
1003700000000691331_11241693.pdf
1003700000000744588_11241856.pdf
1003700000000605654_11240741.pdf
1003700000000728037_11241768.pdf
1003700000000605654_11240741.pdf
1003700000000691623_11241129.pdf
1003700000000689608_11241122.pdf
1003700000000689608_11241122.pdf
1003700000

1003700000000726683_11246352.pdf
1003700000000726683_11246352.pdf
1003700000000726683_11246352.pdf
1003700000000726568_11246349.pdf
1003700000000726568_11246349.pdf
1003700000000728059_11246373.pdf
1003700000000641823_11244651.pdf
1003700000000726683_11246352.pdf
1003700000000641823_11244651.pdf
1003700000000726568_11246349.pdf
1003700000000635737_11244647.pdf
1003700000000726568_11246349.pdf
1003700000000726568_11246349.pdf
1003700000000635698_11245282.pdf
1003700000000726568_11246349.pdf
1003700000000728059_11246373.pdf
1003700000000726568_11246349.pdf
1003700000000724347_11246344.pdf
1003700000000728059_11246373.pdf
1003700000000724347_11246344.pdf
1003700000000724347_11246344.pdf
1003700000000724347_11246344.pdf
1003700000000724347_11246344.pdf
1003700000000723394_11246338.pdf
1003700000000724347_11246344.pdf
1003700000000723394_11246338.pdf
1003700000000723394_11246338.pdf
1003700000000723394_11246338.pdf
1003700000000723394_11246338.pdf
1003700000000723394_11246338.pdf
1003700000

1003700000000692674_11246224.pdf
1003700000000630800_11246126.pdf
1003700000000630800_11246126.pdf
1003700000000630800_11246126.pdf
1003700000000681574_11246196.pdf
1003700000000698071_11247120.pdf
1003700000000698071_11247120.pdf
1003700000000698071_11247120.pdf
1003700000000697219_11247118.pdf
1003700000000697219_11247118.pdf
1003700000000648353_11246162.pdf
1003700000000678722_11247103.pdf
1003700000000648353_11246162.pdf
1003700000000648353_11246162.pdf
1003700000000648353_11246162.pdf
1003700000000697219_11247118.pdf
1003700000000697219_11247118.pdf
1003700000000698071_11247120.pdf
1003700000000678722_11247103.pdf
1003700000000644930_11246152.pdf
1003700000000678722_11247103.pdf
1003700000000678722_11247103.pdf
1003700000000678722_11247103.pdf
1003700000000642168_11247079.pdf
1003700000000678722_11247103.pdf
1003700000000642168_11247079.pdf
1003700000000698071_11247120.pdf
1003700000000644930_11246152.pdf
1003700000000642168_11247079.pdf
1003700000000697219_11247118.pdf
1003700000

1003700000000697811_11249401.pdf
1003700000000675009_11251999.pdf


In [ ]:
# Save generated label file
modified_df.to_csv("batch3_label_revised.csv", index=False)

In [9]:
# Cneck distribution of labels in page level
real_label = modified_df['class']
real_label.value_counts()

flowsheets                97615
progress_notes            63926
mar                       46243
labs                      43658
consult                   18063
history_physical          15633
radiology                 15390
emergency_department      14605
discharge_summary         13080
admission_order            9082
admission_reason           6782
admit_type                 6162
procedures                 3535
discharge_disposition      1266
case_management            1048
physician_query             932
nutritional_assessment      763
Name: class, dtype: int64